In [ ]:
import util.util as util
import re
import tldextract
import re
import importlib
#importlib.reload(util.util)

# Thoughts
# parse Domains like done for the thesis
# parse data from requests file
# most interesting from requests, flags, headers and maybe trying to detect payload

In [ ]:
# URI schemes: http://www.iana.org/assignments/uri-schemes/uri-schemes.xhtml
validURISchemes = ["aaa:", "aaas:", "about:", "acap:", "acct:", "acd:", "acr:", "adiumxtra:", "adt:", "afp:", "afs:",
                   "aim:", "amss:", "android:", "appdata:", "apt:", "ar:", "ark:", "attachment:", "aw:", "barion:",
                   "beshare:", "bitcoin:", "bitcoincash:", "blob:", "bolo:", "browserext:", "cabal:", "calculator:",
                   "callto:", "cap:", "cast:", "casts:", "chrome:", "chrome-extension:", "cid:", "coap:", "coap+tcp:",
                   "coap+ws:", "coaps:", "coaps+tcp:", "coaps+ws:", "com-eventbrite-attendee:", "content:",
                   "content-type:", "crid:", "cvs:", "dab:", "dat:", "data:", "dav:", "diaspora:", "dict:", "did:",
                   "dis:", "dlna-playcontainer:", "dlna-playsingle:", "dns:", "dntp:", "doi:", "dpp:", "drm:", "drop:",
                   "dtmi:", "dtn:", "dvb:", "dvx:", "dweb:", "ed2k:", "elsi:", "embedded:", "ens:", "ethereum:",
                   "example:", "facetime:", "fax:", "feed:", "feedready:", "fido:", "file:", "filesystem:", "finger:",
                   "first-run-pen-experience:", "fish:", "fm:", "ftp:", "fuchsia-pkg:", "geo:", "gg:", "git:",
                   "gizmoproject:", "go:", "gopher:", "graph:", "gtalk:", "h323:", "ham:", "hcap:", "hcp:", "http:",
                   "https:", "hxxp:", "hxxps:", "hydrazone:", "hyper:", "iax:", "icap:", "icon:", "im:", "imap:",
                   "info:", "iotdisco:", "ipfs:", "ipn:", "ipns:", "ipp:", "ipps:", "irc:", "irc6:", "ircs:", "iris:",
                   "iris.beep:", "iris.lwz:", "iris.xpc:", "iris.xpcs:", "isostore:", "itms:", "jabber:", "jar:",
                   "jms:", "keyparc:", "lastfm:", "lbry:", "ldap:", "ldaps:", "leaptofrogans:", "lorawan:", "lvlt:",
                   "magnet:", "mailserver:", "mailto:", "maps:", "market:", "matrix:", "message:",
                   "microsoft.windows.camera:", "microsoft.windows.camera.multipicker:",
                   "microsoft.windows.camera.picker:", "mid:", "mms:", "modem:", "mongodb:", "moz:", "ms-access:",
                   "ms-browser-extension:", "ms-calculator:", "ms-drive-to:", "ms-enrollment:", "ms-excel:",
                   "ms-eyecontrolspeech:", "ms-gamebarservices:", "ms-gamingoverlay:", "ms-getoffice:", "ms-help:",
                   "ms-infopath:", "ms-inputapp:", "ms-lockscreencomponent-config:", "ms-media-stream-id:",
                   "ms-meetnow:", "ms-mixedrealitycapture:", "ms-mobileplans:", "ms-officeapp:", "ms-people:",
                   "ms-project:", "ms-powerpoint:", "ms-publisher:", "ms-restoretabcompanion:", "ms-screenclip:",
                   "ms-screensketch:", "ms-search:", "ms-search-repair:", "ms-secondary-screen-controller:",
                   "ms-secondary-screen-setup:", "ms-settings:", "ms-settings-airplanemode:", "ms-settings-bluetooth:",
                   "ms-settings-camera:", "ms-settings-cellular:", "ms-settings-cloudstorage:",
                   "ms-settings-connectabledevices:", "ms-settings-displays-topology:", "ms-settings-emailandaccounts:",
                   "ms-settings-language:", "ms-settings-location:", "ms-settings-lock:",
                   "ms-settings-nfctransactions:", "ms-settings-notifications:", "ms-settings-power:",
                   "ms-settings-privacy:", "ms-settings-proximity:", "ms-settings-screenrotation:", "ms-settings-wifi:",
                   "ms-settings-workplace:", "ms-spd:", "ms-stickers:", "ms-sttoverlay:", "ms-transit-to:",
                   "ms-useractivityset:", "ms-virtualtouchpad:", "ms-visio:", "ms-walk-to:", "ms-whiteboard:",
                   "ms-whiteboard-cmd:", "ms-word:", "msnim:", "msrp:", "msrps:", "mss:", "mt:", "mtqp:", "mumble:",
                   "mupdate:", "mvn:", "news:", "nfs:", "ni:", "nih:", "nntp:", "notes:", "num:", "ocf:", "oid:",
                   "onenote:", "onenote-cmd:", "opaquelocktoken:", "openpgp4fpr:", "otpauth:", "pack:", "palm:",
                   "paparazzi:", "payment:", "payto:", "pkcs11:", "platform:", "pop:", "pres:", "prospero:", "proxy:",
                   "pwid:", "psyc:", "pttp:", "qb:", "query:", "quic-transport:", "redis:", "rediss:", "reload:",
                   "res:", "resource:", "rmi:", "rsync:", "rtmfp:", "rtmp:", "rtsp:", "rtsps:", "rtspu:", "sarif:",
                   "secondlife:", "secret-token:", "service:", "session:", "sftp:", "sgn:", "shc:", "shttp (OBSOLETE):",
                   "sieve:", "simpleledger:", "simplex:", "sip:", "sips:", "skype:", "smb:", "smp:", "sms:", "smtp:",
                   "snews:", "snmp:", "soap.beep:", "soap.beeps:", "soldat:", "spiffe:", "spotify:", "ssb:", "ssh:",
                   "steam:", "stun:", "stuns:", "submit:", "svn:", "swh:", "swid:", "swidpath:", "tag:", "teamspeak:",
                   "tel:", "teliaeid:", "telnet:", "tftp:", "things:", "thismessage:", "tip:", "tn3270:", "tool:",
                   "turn:", "turns:", "tv:", "udp:", "unreal:", "urn:", "ut2004:", "uuid-in-package:", "v-event:",
                   "vemmi:", "ventrilo:", "ves:", "videotex:", "vnc:", "view-source:", "vscode:", "vscode-insiders:",
                   "vsls:", "wais:", "wcr:", "webcal:", "wifi:", "wpid:", "ws:", "wss:", "wtai:", "wyciwyg:", "xcon:",
                   "xcon-userid:", "xfire:", "xmlrpc.beep:", "xmlrpc.beeps:", "xmpp:", "xri:", "ymsgr:", "z39.50:",
                   "z39.50r:", "z39.50s:"]


localNetwork = re.compile(
    "(.*192\.168\.\d\d?\d?\.\d\d?\d?.*)|(.*10\.\d\d?\d?\.\d\d?\d?\.\d\d?\d?.*)|(.*172\.1[6-9]\.\d\d?\d?\.\d\d?\d?.*)|(.*172\.2[0-9]\.\d\d?\d?\.\d\d?\d?.*)|(.*172\.3[0-1]\.\d\d?\d?\.\d\d?\d?.*)|(.*[^:]fromUI\.local[:\/ ].*)")

def isRemote(v):
    extracted = tldextract.extract(v)
    return extracted.domain != None and extracted.suffix != None and extracted.domain != "" and extracted.suffix != ""


def isLocalNetwork(d):
    if localNetwork.match(d):
        return True

    return False


def getProtocolsPerValuePoint(data, localRemote: bool = True, local: bool = False):
    result = {}
    if 'ValuePoints' not in data:
        return result

    for valuePoint in data['ValuePoints']:
        valueSet = None
        if not 'ValueSet' in valuePoint:
            continue

        schemesFound = set()
        for valueSet in valuePoint['ValueSet']:

            if len(valueSet) == 0:
                continue

            for key in valueSet.keys():
                for value in valueSet[key]:
                    for comma in value.split(","):
                        for v in comma.split('||'):
                            v = v.lower()
                            if not localRemote:
                                if local and not (isLocalNetwork(v) or ('UsesPotentiallyUPnP' in valuePoint and not isRemote(v))):
                                    continue
                                elif not local and (isLocalNetwork(v) or ('UsesPotentiallyUPnP' in valuePoint and not isRemote(v))):
                                    continue

                            for scheme in validURISchemes:
                                if scheme in v:
                                    schemesFound.add(scheme)

            for scheme in schemesFound:
                 result[scheme] = result.get(scheme, 0) + 1


    return result

In [ ]:
def getLocalAddresses(domains):
    result = set()
    for d in domains:
        if util.isLocal(d):
            if '.local' in d:
                if 'UI.local':
                    result.add('fromUI.local')
            elif d.startswith('127.') or d.startswith('0.'):
                continue
            else:
                result.add(d)

    return result


def hasValuePoints(jsonData, localRemote: bool = True, local: bool = False, protocol:bool = False):
    if localRemote:
        return 'ValuePoints' in jsonData and len(jsonData['ValuePoints']) > 0

    domains = util.getUniqueDomainsFromJson(jsonData)
    localAddresses = getLocalAddresses(domains)
    if local:
        return len(localAddresses) > 0

    if protocol:
        protocols = set()
        for d in domains:
            extracted = tldextract.extract(d)
            if isRemote(d) or util.ip.match(d):
                protocols.add(d)
        domains = protocols

    return len(domains) > 0 and len(domains) > len(localAddresses)


def hasMQTT(app, localRemote: bool = True, local: bool = False):
    return hasValuePoints(app.mqtt, localRemote, local, True)


def hasAMQP(app, localRemote: bool = True, local: bool = False):
    return hasValuePoints(app.amqp, localRemote, local, True)


def hasCoap(app, localRemote: bool = True, local: bool = False):
    return hasValuePoints(app.coap, localRemote, local, True)


def hasXMPP(app, localRemote: bool = True, local: bool = False):
    return hasValuePoints(app.xmpp, localRemote, local, True)


In [ ]:
other = ["other", "aaa:", "attachment:", "aw:", "bitcoin:", "chrome:", "cid:", "dab:", "dav:", "did:", "dns:", "ens:", "feed:", "finger:", "geo:", "mailto:", "market:", "nfs:", "pack:", "proxy:", "sip:", "spotify:", "submit:", "turn:", "udp:", "ves:",  "wifi:",'cap:', 'tel:','session:', 'ni:', 'sms:','service:', 'res:', 'about:', 'info:', 'resource:', "ldap:","ssh:",'data:', 'im:', 'icon:',  'content-type:', 'appdata:', 'platform:', 'content:', 'num:', 'mid:', 'mt:', 'go:', 'oid:', "tag:", "ar:"]

def mergeDicts(dict1, dict2):
    result = {}
    dictKeys = set()
    dictKeys.update(dict1.keys())
    dictKeys.update(dict2.keys())
    for key in dictKeys:
        if key in other:
            result["other"] = dict1.get("other", 0) + dict2.get(key, 0)
        else:
            result[key] = dict1.get(key, 0) + dict2.get(key, 0)
    return result


def countSchemeOccurences(dataset):
    result = {}
    for app in dataset:
        hasOther = False
        for key in app.keys():
            if key in other:
                hasOther = True
            else:
                result[key] = result.get(key, 0) + 1
        if hasOther:
            result["other"] = result.get("other", 0) + 1

    return result


def mergeAll(datasetResults):
    result = {}
    for uri in datasetResults:
        result = mergeDicts(result, uri)
    return result


def getAllURIsApp(app, localRemote: bool = True, local: bool = False):
    result = {}
    result = mergeDicts(result, getProtocolsPerValuePoint(app.amqp, localRemote, local))
    result = mergeDicts(result, getProtocolsPerValuePoint(app.coap, localRemote, local))
    result = mergeDicts(result, getProtocolsPerValuePoint(app.endpoints, localRemote, local))
    result = mergeDicts(result, getProtocolsPerValuePoint(app.mqtt, localRemote, local))
    result = mergeDicts(result, getProtocolsPerValuePoint(app.xmpp, localRemote, local))
    result = mergeDicts(result, getProtocolsPerValuePoint(app.udp, localRemote, local))
    result = mergeDicts(result, getProtocolsPerValuePoint(app.webview, localRemote, local))

    return result


def getStatsApp(app, localRemote: bool = True, local: bool = False):
    result = getAllURIsApp(app, localRemote, local)
    if hasMQTT(app, localRemote, local):
        result['MQTT'] = True
    if hasAMQP(app,localRemote, local):
        result['AMQP'] = True
    if hasCoap(app,localRemote, local):
        result['COAP'] = True
    if hasXMPP(app,localRemote, local):
        result['XMPP'] = True
    return result


In [ ]:
def getResultsDataset(dataset, localRemote: bool = True, local: bool = False):
    result = []
    for app in dataset:
        result.append(getStatsApp(app, localRemote, local))

    return result

In [ ]:
# TODO: set the path to your result folder
# set the first argument to the base path of the result folder, second argument is th folder name, third argument a mapping file - that tells for apps in multiple sub datasets which app to include

verDataset = util.get_verified_dataset("", "", "mapping.json")
resultsVer = getResultsDataset(verDataset)
resultsVerLocal = getResultsDataset(verDataset, False, True)
resultsVerRemote = getResultsDataset(verDataset, False, False)



In [ ]:
countedOccVer = countSchemeOccurences(resultsVer)
mergedVer = mergeAll(resultsVer)



In [ ]:
%store verDataset
%store resultsVer
%store resultsVerLocal
%store resultsVerRemote

In [ ]:
# TODO: set the path to load your data
genDataset = util.loadAllData("", True)


In [ ]:
resultsGen = getResultsDataset(genDataset)
resultsGenLocal = getResultsDataset(genDataset, False, True)
resultsGenRemote = getResultsDataset(genDataset, False, False)


In [ ]:
countedOccGen = countSchemeOccurences(resultsGen)
mergedGen = mergeAll(resultsGen)


In [ ]:
%store genDataset
%store resultsGen
%store resultsGenLocal
%store resultsGenRemote


In [ ]:


%store -r genDataset
%store -r resultsGen
%store -r resultsGenLocal
%store -r resultsGenRemote


%store  -r verDataset
%store  -r resultsVer
%store  -r resultsVerLocal
%store  -r resultsVerRemote


In [ ]:
mergedGenLocal = mergeAll(resultsGenLocal)
mergedGenRemote = mergeAll(resultsGenRemote)

countedGenLocal = countSchemeOccurences(resultsGenLocal)
countedGenRemote = countSchemeOccurences(resultsGenRemote)


In [ ]:
mergedVerLocal = mergeAll(resultsVerLocal)
mergedVerRemote = mergeAll(resultsVerRemote)


In [ ]:

countedVerLocal = countSchemeOccurences(resultsVerLocal)
countedVerRemote = countSchemeOccurences(resultsVerRemote)

In [ ]:
countedGenLocal

In [ ]:
import pandas as pd

data = []
keys = set()


for x in countedGenLocal.keys():
    keys.add(x)

for x in countedVerLocal.keys():
    keys.add(x)


for x in countedGenRemote.keys():
    keys.add(x)

for x in countedVerRemote.keys():
    keys.add(x)


for key in keys:
    row = []
    row.append(key)

    row.append(mergedVerLocal.get(key,0))
    row.append(countedVerLocal.get(key,0))

    row.append(mergedGenLocal.get(key,0))
    row.append(countedGenLocal.get(key,0))


    row.append(mergedVerRemote.get(key,0))
    row.append(countedVerRemote.get(key,0))


    row.append(mergedGenRemote.get(key,0))
    row.append(countedGenRemote.get(key,0))



    data.append(row)





df = pd.DataFrame(data, columns = ["scheme","Amount IoT-VER local", "Apps IoT-VER local", "Amount GP-2022 local", "Apps GP-2022 local" , "Amount IoT-VER remote", "Apps IoT-VER remote","Amount GP-2022 remote", "Apps GP-2022 remote"])

In [ ]:
df

In [ ]:
df.to_csv('protocols_ccs.csv', index=False)

In [ ]:
def getOcc(dataset):
    result = 0
    for app in dataset:
        if len(app) > 0:
            result = result + 1

    return result

In [ ]:
df = df.set_index('scheme')

In [ ]:
toRemove = ['unknown', 'tip:', "dweb:", "message:", "query:", "reload:", 'COAP'] # 'android:', 'market:','irc:','wifi:', tooltip - tip


In [ ]:
iotRelatedOther = ["vnc:", "ipps:", "rtmp:", 'palm:', 'pttp:', 'rtsp:', 'rtsps:', 'coap:', 'coaps:','cast:','ipp:','tv:','urn:', 'irc:',"vnc:", "rtmp:" ]

In [ ]:
dfToRemove = df.loc[df.index.isin(toRemove)]

In [ ]:
dfCleared = df.drop(df.loc[df.index.isin(toRemove)].index)

In [ ]:
#dfOther = df.loc[df.index.isin(other)]

In [ ]:
dfOtherIoT = df.loc[df.index.isin(iotRelatedOther)]

In [ ]:
#dfWithoutOther = dfCleared.drop(dfCleared.loc[dfCleared.index.isin(other)].index)
dfWithoutOtherIoT = dfCleared.drop(dfCleared.loc[dfCleared.index.isin(iotRelatedOther)].index)

In [ ]:
dfWithoutOtherIoT

In [ ]:
dfOtherIoT

In [ ]:
dfOtherIoT.sum()

In [ ]:
dfWithoutOtherIoT.loc['IoT related'] = dfOtherIoT.sum()

In [ ]:
dfWithoutOtherIoT

In [ ]:
notToAdd = ['unknown', "dweb:", "message:", "query:", "reload:"] # 'android:', 'market:','irc:','wifi:',


def isLocalCommunication(app):
    localExceptions = ["com.sinepulse.greenhouse.40.apk", "us.originally.rm_trial.2602.apk", "com.triplus.android.client.v2.zipato.1804101201.apk", "com.cairos.automation.14.apk", "air.SmartLog.android.1000109.apk", "com.skybell.app.222.apk", "com.ikea.tradfri.lighting.261.apk", "com.securenettech.smartlink6production.87.apk", "us.originally.rm_trial.apk", "com.triplus.android.client.v2.zipato.apk", "air.SmartLog.android.apk", "com.skybell.app.apk", "com.ikea.tradfri.lighting.apk", "com.securenettech.cove6production.apk", "us.originally.rm_trial.2602.apk", "com.cairos.automation.14.apk", "air.SmartLog.android.1000080.apk", "com.triplus.android.client.v2.zipato.1804101201.apk", "com.ikea.tradfri.lighting.30.apk", "co.candyhouse.sesame.152.apk"]

    for x in localExceptions:
        if x in app.mqtt['pname']:
            return True
    return False

def getNumAppsWithProtocols(results, dataset,local):
    count = 0
    for i in range(0, len(results)):
        x = results[i]
        for y in x.keys():
            if(local and (y not in toRemove or isLocalCommunication(dataset[i]))) or (not local and y not in notToAdd):
                count = count + 1
                break
#            else:
#                print(y)
    return count


In [ ]:
numAppsLocalGen = getNumAppsWithProtocols(resultsGenLocal,genDataset ,True)
numAppsLocalVer = getNumAppsWithProtocols(resultsVerLocal,verDataset ,True)


numAppsRemoteGen = getNumAppsWithProtocols(resultsGenRemote,genDataset ,False)
numAppsRemoteVer = getNumAppsWithProtocols(resultsVerRemote,verDataset ,False)


In [ ]:
numAppsLocalGen

In [ ]:
numAppsLocalVer

In [ ]:
numAppsRemoteGen

In [ ]:
numAppsRemoteVer

In [ ]:
dfWithoutOtherIoT['Apps GP-2022 local percentage'] = dfWithoutOtherIoT['Apps GP-2022 local'].divide(numAppsLocalGen)
dfWithoutOtherIoT['Apps IoT-VER local percentage'] = dfWithoutOtherIoT['Apps IoT-VER local'].divide(numAppsLocalVer)


dfWithoutOtherIoT['Apps GP-2022 remote percentage'] = dfWithoutOtherIoT['Apps GP-2022 remote'].divide(numAppsRemoteGen)
dfWithoutOtherIoT['Apps IoT-VER remote percentage'] = dfWithoutOtherIoT['Apps IoT-VER remote'].divide(numAppsRemoteVer)


In [ ]:
dfWithoutOtherIoT

In [ ]:
dfWithoutOtherIoT.to_csv('protocols_processed_ccs.csv')

In [ ]:
dfOtherIoT['Apps GP-2022 local percentage'] = dfOtherIoT['Apps GP-2022 local'].divide(numAppsLocalGen)
dfOtherIoT['Apps IoT-VER local percentage'] = dfOtherIoT['Apps IoT-VER local'].divide(numAppsLocalVer)


dfOtherIoT['Apps GP-2022 remote percentage'] = dfOtherIoT['Apps GP-2022 remote'].divide(numAppsRemoteGen)
dfOtherIoT['Apps IoT-VER remote percentage'] = dfOtherIoT['Apps IoT-VER remote'].divide(numAppsRemoteVer)

dfOtherIoT.to_csv('protocolsIoT_ccs.csv')

In [ ]:
dfAll = dfWithoutOtherIoT.sort_index()

In [ ]:
dfWithoutOtherIoT

In [ ]:
dfOtherIoT = dfOtherIoT.sort_index()

In [ ]:
dfAll = pd.concat([dfAll,dfOtherIoT ])

In [ ]:
#dfAll = dfAll.style.format({'Apps 2018 local percentage': '{:.2%}','Apps August 2021 local percentage': '{:.2%}','Apps January 2022 local percentage': '{:.2%}', 'Apps 2018 remote percentage': '{:.2%}','Apps August 2021 remote percentage': '{:.2%}','Apps January 2022 remote percentage': '{:.2%}' })

In [ ]:
dfAll['Apps GP-2022 local percentage'] = dfAll['Apps GP-2022 local percentage'].map(lambda n: '{:,.2%}'.format(n))
dfAll['Apps IoT-VER local percentage'] = dfAll['Apps IoT-VER local percentage'].map(lambda n: '{:,.2%}'.format(n))


dfAll['Apps GP-2022 remote percentage'] = dfAll['Apps GP-2022 remote percentage'].map(lambda n: '{:,.2%}'.format(n))
dfAll['Apps IoT-VER remote percentage'] = dfAll['Apps IoT-VER remote percentage'].map(lambda n: '{:,.2%}'.format(n))


In [ ]:
indexes = []
data = {}
for index, row in dfAll.iterrows():
    indexes.append(index)

    g2022local =  data.get('GP-2022-local', [])
    g2022local.append(f"{row['Amount GP-2022 local']}-{row['Apps GP-2022 local']}({row['Apps GP-2022 local percentage']})")
    data['GP-2022-local'] = g2022local

    verlocal =  data.get('IoT-VER-local', [])
    verlocal.append(f"{row['Amount IoT-VER local']}-{row['Apps IoT-VER local']}({row['Apps IoT-VER local percentage']})")
    data['IoT-VER-local'] = verlocal


    g2022remote =  data.get('GP-2022-remote', [])
    g2022remote.append(f"{row['Amount GP-2022 remote']}-{row['Apps GP-2022 remote']}({row['Apps GP-2022 remote percentage']})")
    data['GP-2022-remote'] = g2022remote

    verremote =  data.get('IoT-VER-remote', [])
    verremote.append(f"{row['Amount IoT-VER remote']}-{row['Apps IoT-VER remote']}({row['Apps IoT-VER remote percentage']})")
    data['IoT-VER-remote'] = verremote


 # Creates pandas DataFrame.



In [ ]:
df = pd.DataFrame(data, index =indexes) #


In [ ]:
df.to_csv('protocols_summarized_ccs.csv')

In [ ]:
# NOTE, we used seperate files to analyze MQTT, XMPP, AMQP, COAP, and used those more precise results.
# That also lead to small changes in percentage

In [ ]:
df

In [ ]:
print(df.to_latex())

In [ ]:
len(data["IoT-VER-local"])

In [ ]:
len(data["GP-2022-remote"])

In [ ]:
len(data["IoT-VER-local"])

In [ ]:
resultsVerLocal

In [ ]:
resultsVerRemote

In [ ]:
def group_other_iot(dataset, names=[]):
    count = 0
    for app in dataset:
        add_other = False
        for value in app.keys():
            for v2 in names:
                if value == v2:
                    print(value)
                    add_other = True

        if add_other:
            count += 1

    return count


In [ ]:
group_other_iot(resultsVerRemote, ["ipp:", "ipps:", "rtmp:", "vnc:"])

In [ ]:
IPP + IPPs + RMTP + VNC

In [ ]:
numAppsRemoteVer

In [ ]:
print(f"{4/7114*100:.2f}")